This tutorial will demonstrate how to create an MDM pipeline using MindPype

In [ ]:
# this is to setup the path so we can import the mindpype library
import os; os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [ ]:
import mindpype as mp
import numpy as np

The first step to creating a pipeline is to create a session, which serves as a sandbox for all components in the pipeline. After creating the session we will create a graph which represents the pipeline.

In [ ]:
# create a session
s = mp.Session.create()
trial_graph = mp.Graph.create(s)

We will create fake data to train our MDM classifier with and store this data in a Tensor using the ```create_from_data()``` factory method.



In [ ]:
# create fake data for training
raw_training_data = np.random.normal(loc=0.0,scale=1.0,size=(180,250,12))
training_data = np.zeros((180,12,12))
for i in range(180):
    training_data[i,:,:] = np.cov(raw_training_data[i,:,:],rowvar=False)

labels = np.asarray([0]*60 + [1]*60 + [2]*60)
X = mp.Tensor.create_from_data(s,training_data)
y = mp.Tensor.create_from_data(s,labels)

Next we will create our input and output data containers for the graph. Our pipeline will take input data and will output the predicted label so we will create a tensor object for our input and a scalar object for our output.

Also, we will create virtual tensors to hold ant intermediate values that are calculated throughout the pipeline using the ```create_virtual()``` method. Since these intermediate values represent data that is only required in the proces of completing a calculation and we do not need to access them later, the virtual type is ideal.

In [ ]:
input_data = np.random.randn(12,500)
t_in = mp.Tensor.create_from_data(s,input_data)
s_out = mp.Scalar.create_from_value(s,-1)

t_virt = [mp.Tensor.create_virtual(s),
            mp.Tensor.create_virtual(s)]

We will create a filter object that we will later pass into our filter node when we add it to our graph.

In [ ]:
# create a filter
order = 4
bandpass = (8,35) # in Hz
fs = 250
f = mp.Filter.create_butter(s,order,bandpass,btype='bandpass',fs=fs,implementation='sos')

Next, we will add our nodes to the graph using the ```add_to_graph()``` faactory method. We first filter the Tensor. Then we will calculate the covariance of the filtered Tensor. Finally, we use a Riemann MDM classifier to obtain a predicted label.

In [ ]:
# add the nodes
mp.kernels.FilterKernel.add_to_graph(trial_graph,t_in,f,t_virt[0])
mp.kernels.CovarianceKernel.add_to_graph(trial_graph,t_virt[0],t_virt[1])
mp.kernels.RiemannMDMClassifierKernel.add_to_graph(trial_graph,
                                                    t_virt[1],
                                                    s_out,3,X,y)


With all of our nodes added to our graph, we will then verify the graph using the ```verify()``` method. Verifying the graph orders the nodes for execution and ensure that the inputs and outputs of each processing node are appropriately typed and sized.

After our graph has been verified, the next step is to initialize the graph using the ```initialize()``` method. This step is required for pipelines that have methods that need to be trained or fit.

In [ ]:
# verify the session (i.e. schedule the nodes)
trial_graph.verify()
# intialize the graph (i.e. train the classifier)
trial_graph.initialize()


Finally, we can run our pipeline by calling the ```execute()``` method on our graph.

In [ ]:
# RUN!
trial_graph.execute(0)

print(s_out.data)